# FTX reader

In [13]:
import matplotlib
from datetime import *
%matplotlib inline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px
import nest_asyncio
nest_asyncio.apply()
import os,sys
sys.path.append('../')
from utils.io_utils import *
from histfeed.ftx_history import main

end_time=datetime.now(timezone.utc)-timedelta(days=1)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(days=60)
halflife=timedelta(days=1)
modes=[('h','high'),('c','close'),('l','low'),('v','size')]

In [2]:
weights = pd.read_excel(os.path.join(os.sep, 'home','david',"config", "pfoptimizer",'current_weights.xlsx')).sort_values(by='optimalWeight',key=lambda f: np.abs(f),ascending=False)
coins = ['CRV']#list(weights.set_index(['name']).drop(['USD','total']).head(5).index.map(lambda f: f.replace('-PERP','')))
weights

,name,spotBenchmark,ExpectedBenchmark,FundingBenchmark,currentWeight,optimalWeight,ExpectedCarry,RealizedCarry,excessIM,excessMM,transactionCost,time
10,USD,NaN,0.026772,NaN,3664.999664,3087.759828,0.000000,0.000000,3042.312035,2.744695e+03,0,2022-06-20 12:00:00
9,YFI-PERP,NaN,-0.000557,0.000312,0.000000,-1028.647082,196.777882,0.000000,-1308.667676,-1.208089e+03,0,2022-06-20 12:00:00
11,total,-0.000469,-0.056811,-0.000144,908.955866,908.955866,475.476082,-4608.458399,0.000010,-3.616738e-07,0,2022-06-20 12:00:00
0,UNI-PERP,NaN,0.000212,0.000432,-124.926300,519.848574,40.253435,-22.449631,467.863717,4.782607e+02,0,2022-06-20 12:00:00
5,SUSHI-PERP,NaN,-0.000618,-0.000552,-425.970900,-426.754429,49.962114,46.721298,-515.474428,-4.754943e+02,0,2022-06-20 12:00:00
6,HT-PERP,-0.000863,-0.000503,-0.000168,0.000000,-426.678498,51.310535,0.000000,-542.829867,-5.011102e+02,0,2022-06-20 12:00:00
8,RAY-PERP,NaN,-0.000654,-0.000048,0.000000,-418.303539,67.572503,0.000000,-562.249169,-5.194346e+02,0,2022-06-20 12:00:00
7,OMG-PERP,-0.000550,-0.000446,0.000096,0.000000,-279.159726,36.555527,0.000000,-375.223515,-3.466507e+02,0,2022-06-20 12:00:00
1,LINK-PERP,0.001239,0.000134,0.001032,-412.692000,196.184051,9.619848,-159.176955,176.565646,1.804893e+02,0,2022-06-20 12:00:00
4,BCH-PERP,-0.001582,-0.000779,-0.000720,0.000000,-170.995874,13.059862,0.000000,-206.545016,-1.905254e+02,0,2022-06-20 12:00:00


In [7]:
futures_df = main([None,'get','ftx','max'])

INFO:histfeed.ftx_history:Logger set
INFO:histfeed.ftx_history:Cannot find the nb_of_days param. Defaulting to nb_of_days=100
INFO:histfeed.ftx_history:histfeed running with params ['get', 'ftx', 'max']
INFO:histfeed.ftx_history:Getting history...


In [8]:
futures_df.filter(like='CRV')


,CRV-PERP_rate_funding,CRV-PERP_mark_o,CRV-PERP_mark_h,CRV-PERP_mark_l,CRV-PERP_mark_c,CRV-PERP_mark_volume,CRV-PERP_indexes_o,CRV-PERP_indexes_h,CRV-PERP_indexes_l,CRV-PERP_indexes_c,...,CRV-PERP_rate_c,CRV-PERP_rate_h,CRV-PERP_rate_l,CRV_price_o,CRV_price_h,CRV_price_l,CRV_price_c,CRV_price_volume,CRV_rate_borrow,CRV_rate_size
2022-03-12 14:00:00,-0.262980,2.00690,2.01220,1.98570,1.99650,9.835621e+06,2.007380,2.011980,1.988263,1.997526,...,-0.187690,0.039852,-0.470751,2.00810,2.01255,1.98720,1.99640,5.027002e+05,999.0,NaN
2022-03-12 15:00:00,-0.227916,1.99650,2.01770,1.99280,2.01015,1.549720e+07,1.997827,2.017131,1.995263,2.011927,...,-0.322676,0.102960,-0.450847,1.99640,2.01710,1.99540,2.01200,9.035553e+05,999.0,NaN
2022-03-12 16:00:00,-0.035064,2.01025,2.01760,2.00395,2.01145,5.919651e+06,2.012861,2.016164,2.004927,2.011131,...,0.057953,0.260079,-0.177981,2.01265,2.01665,2.00500,2.01100,5.868444e+05,999.0,NaN
2022-03-12 17:00:00,-0.017532,2.01145,2.02650,2.00690,2.01235,1.192310e+07,2.011198,2.024333,2.007891,2.012625,...,-0.049856,0.391031,-0.180245,2.01100,2.02520,2.00695,2.01250,5.040331e+05,999.0,NaN
2022-03-12 18:00:00,-0.078894,2.01235,2.02090,2.00695,2.01780,1.051596e+07,2.012858,2.021392,2.010084,2.020225,...,-0.438475,-0.088967,-0.569550,2.01250,2.02135,2.00810,2.01885,1.236723e+06,999.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-20 09:00:00,-0.227916,0.66330,0.67625,0.66330,0.67385,9.091121e+06,0.664243,0.675048,0.664243,0.673403,...,0.242297,0.650453,-0.518279,0.66310,0.67675,0.66310,0.67355,3.897391e+05,999.0,NaN
2022-06-20 10:00:00,-0.122724,0.67540,0.69890,0.67100,0.69705,4.343758e+07,0.675181,0.698258,0.671872,0.696441,...,0.319295,0.335706,-0.474080,0.67675,0.69855,0.67275,0.69730,1.929642e+06,999.0,NaN
2022-06-20 11:00:00,-0.271746,0.69705,0.70085,0.69145,0.69560,5.005303e+06,0.698772,0.699751,0.692213,0.694064,...,0.808505,0.573660,-0.402760,0.69730,0.70060,0.69265,0.69520,3.580108e+05,999.0,NaN
2022-06-20 12:00:00,-0.070128,0.70815,0.70855,0.68580,0.69355,4.013270e+07,0.708369,0.708369,0.687175,0.694707,...,-0.608567,0.093560,-0.730721,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### rates markets

In [14]:
selected = [df for df in futures_df.columns if any(df.split('-')[0].split('_')[0]==coin for coin in (coins+['USD']))]
prices=futures_df[[c for c in selected if any(string in c for string in ['price_c'])]]

short_rates=futures_df[[c for c in selected if any(string in c for string in ['PERP_rate_c','rate_borrow'])]]
short_rates=short_rates.ewm(halflife=halflife,times=short_rates.index).mean()

borrow_size=futures_df[[c for c in selected if any(string in c for string in ['_rate_size','price_volume'])]]

for c in coins:
    long_perp_carry=short_rates[c+'-PERP_rate_c']-short_rates['USD_rate_borrow']
    short_perp_carry=-short_rates[c+'-PERP_rate_c']-short_rates[c+'_rate_borrow']
    short_rates[c+'_best_perp_carry']= pd.concat([long_perp_carry, short_perp_carry]).max(level=0)
   # long_rates[c+'_long_future_carry']=long_rates[c+'-1231_rate_c']-short_rates['USD_rate_borrow']
   # long_rates[c+'_short_future_carry']=-long_rates[c+'-1231_rate_c']-short_rates[c+'_rate_borrow']

short_rates
pd.concat([short_rates],axis=0).iplot()#))

/tmp/ipykernel_48751/1800538636.py:12: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().



In [15]:
pd.concat([short_rates.filter(like='borrow')],axis=0).iplot()#))

In [17]:
pd.concat([borrow_size],axis=0).iplot()#))

### carry directionality

In [ ]:
plotthat = futures_df[['BNB-PERP/rate/h',
       'BNB-PERP/rate/l', 'BNB-PERP/rate/c']]
plotthat.columns=['h','l','c']

In [ ]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=plotthat.index,#+x_rev,
    y=0.5*(plotthat['l'].values+plotthat['h'].values),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='BNB',
))
fig.add_trace(go.Scatter(
    x=plotthat.index, y=plotthat['c'].values,
    line_color='rgb(0,100,80)',
    name='Fair',
))

fig.update_traces(mode='lines')
fig.show()

In [ ]:
pd.concat([prices,short_rates.filter(like='carry')],axis=0).iplot(secondary_y=list(prices.columns))

# stopout: show min/max of premiums

In [ ]:
end_time=datetime.now(timezone.utc)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(hours=1)
granular_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='15s',coin_list=coins)

In [ ]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compute_premium_vs_spot(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/price/'+modes[mode][0]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [ ]:
granular_df['premium']=compute_premium_vs_spot(granular_df)
prices=granular_df[[c+'/price/c' for c in coins]]
granular_df[['premium']].iplot()

In [ ]:
premium=compute_premium(granular_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
pd.concat([prices,ewMax],axis=0).iplot(secondary_y=list(prices.columns))

In [ ]:
premium=compute_premium(granular_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
pd.concat([prices,ewMin],axis=0).iplot(secondary_y=list(prices.columns))